<p style="align: center;"><img src="https://raw.githubusercontent.com/dimkablin/SegmentationSM/master/src/logo2.jpg", width=550, height=300></p>

<h1 style="text-align: center;"><b>Backend SM</b></h1>
<h2 style="text-align: center;"><b>Поиск архитекутуры для сегментации объекта для проекта SM </b></h2>

In [5]:
import os
from LookGenerator.config.config import DatasetConfig

config = DatasetConfig(os.environ)
config.show_details()

DATASET_DIR
BACKGROUND_DATASET


## dataloader init

In [ ]:
from LookGenerator.datasets.background_cut_dataset import PersonDataset

person_dataset =  PersonDataset(config.DATASET_DIR, "image", "image-parse-v3",
                                background_root_dir=config.BACKGROUND_DATASET,
                                dir_name_background="ScreenShot")

In [ ]:
from skimage.transform import resize

size = (276, 276) # ???


## train loop

# Architectures
## 1. our Unet

## 2. SegNet

## 3. a few Unet's arhc